![xleaflet](images/xleaflet.png)

<center> <h1>C++ backend for the jupyter-leaflet map visualization library</h1> </center>

<center> <h1>Basic usage</h1> </center>

In [ ]:
#include <iostream>
#include <string>

#include "xleaflet/xmap.hpp"
#include "xleaflet/xtile_layer.hpp"
#include "xleaflet/xmarker.hpp"
#include "xleaflet/xmarker_cluster.hpp"
#include "xleaflet/xbasemaps.hpp"

#include "xwidgets/xbox.hpp"
#include "xwidgets/xtext.hpp"
#include "xwidgets/xlabel.hpp"

## Create a ``map`` widget with some ``text`` widgets

In [ ]:
auto map = xlf::map::initialize()
    .center({50, 354})
    .zoom(5)
    .finalize();

xw::vbox vbox;
xw::hbox hbox1, hbox2;
xw::label label1, label2;
xw::text center, mouse_position;

center.disabled = true;
mouse_position.disabled = true;

vbox.add(map);
vbox.add(hbox1);
vbox.add(hbox2);

label1.value = "Center:";
center.value = "[50.0, 354.0]";
hbox1.add(label1);
hbox1.add(center);

label2.value = "Mouse position:";
mouse_position.value = "Mouse out";
hbox2.add(label2);
hbox2.add(mouse_position);

vbox

## Observer on ``map.center``

In [ ]:
void update_center(xlf::map& map)
{
    std::string lat = std::to_string(map.center().front());
    std::string lng = std::to_string(map.center().back());
    
    center.value = "[" + lat + ", " + lng + "]";
}

XOBSERVE(map, center, update_center);

In [ ]:
void update_mouse_position(nl::json event)
{
    if (event["type"] == "mousemove")
    {
        mouse_position.value = event["coordinates"].dump();
    }
    
    if (event["type"] == "mouseout")
    {
        mouse_position.value = "Mouse out";
    }
}

map.on_interaction(update_mouse_position);

## Display available base maps

In [ ]:
xlf::basemaps().dump(6)

## Create a ``tile_layer``

In [ ]:
auto nasa_layer = xlf::basemap({"NASAGIBS", "ModisTerraTrueColorCR"}, "2017-04-08");
map.add_layer(nasa_layer);

## Create a ``marker_cluster``

In [ ]:
auto marker1 = xlf::marker::initialize()
    .location({50, 354})
    .finalize();
auto marker2 = xlf::marker::initialize()
    .location({52, 356})
    .finalize();
auto marker3 = xlf::marker::initialize()
    .location({48, 352})
    .finalize();
auto marker_cluster = xlf::marker_cluster::initialize()
    .markers({marker1, marker2, marker3})
    .finalize();

map.add_layer(marker_cluster);

## Remove a layer from the ``map``

In [ ]:
map.remove_layer(marker_cluster);

## Create a single custom ``marker`` with a ``on_move`` callback

In [ ]:
using size_type = std::array<int, 2>;
size_type icon_size({38, 95});
size_type icon_anchor({22, 94});

auto icon = xlf::icon::initialize()
    .icon_url("https://leafletjs.com/examples/custom-icons/leaf-red.png")
    .icon_size(icon_size)
    .icon_anchor(icon_anchor)
    .finalize();

auto marker = xlf::marker::initialize()
    .location({50, 356})
    .icon(icon)
    .rotation_angle(90)
    .rotation_origin("22px 94px")
    .finalize();

map.add_layer(marker);

In [ ]:
void print_location(nl::json event)
{
    std::cout << "marker location: (" << event.find("location").value().at(0) 
        << ", " << event.find("location").value().at(1) << ")" << std::endl;
}

marker.on_move(print_location);